# lab5

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

sns.set_style("darkgrid")
%matplotlib inline

In [2]:
df = pd.read_csv("Data.csv")

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
df.dropna(inplace=True)
df["Customer ID"] = df["Customer ID"].astype(int)

## Интересующие события и события наблюдения

в качестве событий будем использовать возвраты товаров

In [5]:
df['is_returned'] = df["Quantity"] < 0

In [18]:
qs = [.25, .5, .75]
price_qs = df["Price"].quantile(qs).values
quantity_qs = df["Quantity"].abs().quantile(qs).values

In [22]:
from functools import partial

def round_to_quantile(qs, x):
    return qs[(np.abs(qs - abs(x))).argmin()]

df["Price_rounded"] = df["Price"].apply(partial(round_to_quantile, price_qs))
df["Quantity_rounded"] = df["Quantity"].apply(partial(round_to_quantile, quantity_qs))

> событие наблюдения: цена и количество между квантилями датасета, а также страна
> 
> интересующее событие: товар вернули

In [21]:
((df["is_returned"] == True)).sum()

17341

## Полное совместное распределение

In [30]:
# Группировка данных по всем колонкам и подсчет частоты каждой уникальной комбинации
frequency_table = df.groupby(['is_returned', 'StockCode', 'Country']).size().reset_index(name='Frequency')

In [31]:
# Сумма всех частот для нормализации
total = np.sum(frequency_table['Frequency'])

# Вычисление вероятности для каждой комбинации
frequency_table['Probability'] = frequency_table['Frequency'] / total

In [32]:
assert np.isclose(frequency_table['Probability'].sum(), 1)

In [33]:
frequency_table

,is_returned,StockCode,Country,Frequency,Probability
0,False,10002,Australia,2,0.000003
1,False,10002,Denmark,3,0.000004
2,False,10002,EIRE,5,0.000007
3,False,10002,France,31,0.000042
4,False,10002,Germany,7,0.000010
...,...,...,...,...,...
30710,True,POST,Spain,4,0.000005
30711,True,POST,Sweden,5,0.000007
30712,True,POST,Switzerland,3,0.000004
30713,True,POST,United Kingdom,101,0.000138


## Причины следствия

> возможно в стране United Kingdom возвращают товары чаще

In [27]:
# частная вероятность
frequency_table_part = df.groupby(['is_returned', 'Country']).size().reset_index(name='Frequency')
total = np.sum(frequency_table_part['Frequency'])
frequency_table_part['Probability'] = frequency_table_part['Frequency'] / total

In [28]:
frequency_table_part

,is_returned,Country,Frequency,Probability
0,False,Australia,1757,0.002398
1,False,Austria,845,0.001153
2,False,Bahrain,59,0.000081
3,False,Belgium,2842,0.003880
4,False,Brazil,94,0.000128
...,...,...,...,...
67,True,Switzerland,51,0.000070
68,True,USA,115,0.000157
69,True,United Arab Emirates,3,0.000004
70,True,United Kingdom,14817,0.020226


In [35]:
# условная вероятность
frequency_table_cond_1 = df[df['Country'] == 'United Kingdom'].groupby(['is_returned', 'StockCode', 'Price_rounded', 'Quantity_rounded']).size().reset_index(name='Frequency')
total = np.sum(frequency_table_cond_1['Frequency'])
frequency_table_cond_1['Probability'] = frequency_table_cond_1['Frequency'] / total
frequency_table_cond_1

,is_returned,StockCode,Price_rounded,Quantity_rounded,Frequency,Probability
0,False,10002,1.25,2.0,76,0.000116
1,False,10002,1.25,6.0,31,0.000047
2,False,10002,1.25,12.0,133,0.000202
3,False,10080,1.25,2.0,9,0.000014
4,False,10080,1.25,6.0,1,0.000002
...,...,...,...,...,...,...
18136,True,PADS,3.75,2.0,1,0.000002
18137,True,POST,1.25,2.0,4,0.000006
18138,True,POST,1.95,2.0,15,0.000023
18139,True,POST,3.75,2.0,82,0.000125


In [36]:
# условная вероятность
frequency_table_cond_2 = df[df['Country'] != 'United Kingdom'].groupby(['is_returned', 'StockCode', 'Price_rounded', 'Quantity_rounded']).size().reset_index(name='Frequency')
total = np.sum(frequency_table_cond_1['Frequency'])
frequency_table_cond_2['Probability'] = frequency_table_cond_2['Frequency'] / total
frequency_table_cond_2

,is_returned,StockCode,Price_rounded,Quantity_rounded,Frequency,Probability
0,False,10002,1.25,2.0,3,0.000005
1,False,10002,1.25,12.0,59,0.000090
2,False,10120,1.25,2.0,1,0.000002
3,False,10120,1.25,12.0,2,0.000003
4,False,10123C,1.25,12.0,5,0.000008
...,...,...,...,...,...,...
8507,True,M,1.95,2.0,2,0.000003
8508,True,M,3.75,2.0,81,0.000123
8509,True,POST,1.25,2.0,1,0.000002
8510,True,POST,3.75,2.0,56,0.000085


## Предсказание событий

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer, make_column_selector

In [42]:
xy = df[['Country', 'Price', 'is_returned', 'Quantity']]
X = df[['Country', 'Price', 'Quantity']].copy()
y = df['is_returned'].copy()

category_columns = ['Country']

X[category_columns] = X[category_columns].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42)

In [43]:
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)

pipeline = make_pipeline(
    ordinal_encoder,
    HistGradientBoostingClassifier(
        categorical_features=[0],
        random_state=43,
    ),
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f9a43689280>)],
                                   verbose_feature_names_out=False)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(categorical_features=[0],
                                                random_state=43))])

In [44]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pipeline.predict(X_test))

1.0